# Скрипт по сбору данных

Данный скрипт - шаблонный для нашей группы.

Мы используем конфигурационные файлы, которые удобно редактировать. почти все библиотеки встроенные.

Файлы проекта (required):

1) .ipynb (парсинг каналов по тематикам и исполнителям - шаблон)
4) config.yml (YAML файл с конфигурацией для создания клиента и создания запроса)
5) Сведенный датасет


## Инициализация скрипта

In [2]:
#1
# Loading Libraries
import pathlib

import pandas as pd
import googleapiclient.discovery as api
import numpy as np
import requests

from tqdm import tqdm

from pathlib import Path
import os
import yaml
import json

In [3]:
#2

# Defining variables

# Determine path to the working directory
path = Path().resolve()
print('Current Path')
print(path)

# Opening and parsing yaml file
if path:

    cnf_path = str(path) + '/' + 'config.yml'

    with open(cnf_path, 'r') as f:
        config = yaml.safe_load(f)

    print('Loaded with Pathlib ')
else:
    try:
        with open('config.yml', 'r') as f:
            config = yaml.safe_load(f)
    except:
        print('Попробуете распаковать проект целиком?')

DATA_PATH = pathlib.Path().resolve() / config['api']['DATA_PATH']

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
print('ПУТЬ ДАННЫХ:', DATA_PATH)

Current Path
/Users/andrey/PycharmProjects/DataMiningProjects/notebooks/HW1
Loaded with Pathlib 
ПУТЬ ДАННЫХ: /Users/andrey/PycharmProjects/DataMiningProjects/notebooks/HW1/data


# Создание клиента запроса

In [13]:
# 6 Определение API-Key Storage

# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)

key_storage = KeyStorage(config['api']['keys'])

# Запрос: поиск бьюти блоггеров
## Парсинг данных из Youtube


In [15]:
#6

# Создаем параметры нашего поиска

query_params = {
    "type": "channel",
    "maxResults": 50,
    "part": "snippet",
    "regionCode": "RU",
}

display(query_params)

{'type': 'channel', 'maxResults': 50, 'part': 'snippet', 'regionCode': 'RU'}

In [16]:
# 7 Создание клиента
api_name = "youtube"
api_version = "v3"
cur_key = key_storage.get_next_key()

def get_api(cur_key: str):
    api_client = api.build(
            api_name,
            api_version,
            developerKey=cur_key
        )
    return api_client


youtube = get_api(cur_key)

query = 'бьюти блогеры'


In [19]:
# 8 Поиск без order

request = youtube.search().list(
        q = query,
        **query_params # using
)
response = request.execute()

df_supplemented = pd.json_normalize(response['items'])

    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            q = query,
            pageToken = response['nextPageToken'],
            **query_params
        )
    response = request.execute()

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])


print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates('id.channelId'))}")


print('Размер получившегося датафрейма', df_supplemented.shape)

Итерация БЕЗ включения аргумента order №0
Итерация БЕЗ включения аргумента order №1
Итерация БЕЗ включения аргумента order №2
Итерация БЕЗ включения аргумента order №3
Итерация БЕЗ включения аргумента order №4
Итерация БЕЗ включения аргумента order №5
Искомых объектов 399 а найденных БЕЗ включения каких-либо значений аргумента order 328
Размер получившегося датафрейма (338, 14)


In [20]:
# 9 Варьирование значения аргумента order:
order_list = ['date', 'rating', 'title', 'videoCount', 'viewCount']

In [21]:
# 10
# Цикл для прохода по значениям аргумента order, внутри которых проход по всем страницам выдачи:
i = 0
for order in order_list:
    # Для остановки алгоритма, если все искомые объекты найдены
    # БЕЗ включения каких-либо значений аргумента order (в т.ч. вообще БЕЗ них):
    if len(df_supplemented.drop_duplicates('id.channelId')) < response['pageInfo']['totalResults']:

        # Первый заход с каждым значение аргумента order:
        request = youtube.search().list(q = query,
                                        order=order,
                                        **query_params
        )
        response = request.execute()

        # Визуализация процесса:
        print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')
        i += 1

        # Занесение собранных данных в итоговую таблицу df_supplemented:
        df = pd.json_normalize(response["items"])
        df_supplemented = pd.concat([df, df_supplemented])

        # Заходы с тем же значением аргумента order на следующие страницы:
        while ('nextPageToken' in response.keys())\
        & (len(df_supplemented.drop_duplicates('id.channelId')) < response['pageInfo']['totalResults'])\
        & (len(response["items"]) > 0):
        # второе условие -- для остановки алгоритма, если все искомые объекты найдены
        # БЕЗ какой-то из следующих страниц ( в т.ч. вообще БЕЗ них)
        # третье условие -- для остановки алгоритма, если предыдущая страница выдачи содержит 0 объектов

            request = youtube.search().list(
                                            q = query,
                                            pageToken = response['nextPageToken'],
                                            **query_params
            )
            response = request.execute()

            # Визуализация процесса:
            print(f'Итерация №{i}, "order" {order}, "items" {len(response["items"])}')

            i += 1

            # Занесение собранных данных в итоговую таблицу df_supplemented:
            df_additional = pd.json_normalize(response["items"])
            df_supplemented = pd.concat([df_supplemented, df_additional])

        # Сохранение составленных для каждого значения аргумента order таблиц в Excel -- на всякий случай:
        df_supplemented.to_excel(DATA_PATH / f'Каналы_Sorted_by_{order}.xlsx', index=False)
    else:
        print('Все искомые объекты найдены БЕЗ включения некоторых значений аргумента order (в т.ч. вообще БЕЗ них)')

Итерация №0, "order" date, "items" 4
Итерация №1, "order" rating, "items" 4
Итерация №2, "order" title, "items" 4
Итерация №3, "order" videoCount, "items" 4
Итерация №4, "order" viewCount, "items" 4


# Просмотр полученного DataFrame и его преобразование

In [22]:
# 11

df_search = df_supplemented.copy(deep=True)
df_search = df_search.drop_duplicates(subset=['id.channelId'],)

df_search.head()


,kind,etag,id.kind,id.channelId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.medium.url,snippet.thumbnails.high.url,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,AEuDg6Fnte0mIzQw9_LOjaGh3Bw,youtube#channel,UCfZBBpWbMLxx_MJs6U2yB2g,2021-06-19T16:12:54Z,UCfZBBpWbMLxx_MJs6U2yB2g,❤️Бьюти Блогеры❤️,,https://yt3.ggpht.com/S1ZsXkLCFV7DMix7BJOcslL6...,https://yt3.ggpht.com/S1ZsXkLCFV7DMix7BJOcslL6...,https://yt3.ggpht.com/S1ZsXkLCFV7DMix7BJOcslL6...,❤️Бьюти Блогеры❤️,none,2021-06-19T16:12:54Z
1,youtube#searchResult,5UGJ_G5MWV86j79xh9961nr3nUc,youtube#channel,UCkb6cE2MJvzNVs_wic2Aipg,2016-12-10T09:36:55Z,UCkb6cE2MJvzNVs_wic2Aipg,Бьюти Блогеры,,https://yt3.ggpht.com/ytc/AL5GRJXSDB42IlNPfgej...,https://yt3.ggpht.com/ytc/AL5GRJXSDB42IlNPfgej...,https://yt3.ggpht.com/ytc/AL5GRJXSDB42IlNPfgej...,Бьюти Блогеры,none,2016-12-10T09:36:55Z
2,youtube#searchResult,PpuKcoyHU6xRUQrCip5SFsLR_wM,youtube#channel,UCZI_KsziUHl_aRUTqOmq3hg,2021-11-30T11:21:15Z,UCZI_KsziUHl_aRUTqOmq3hg,Бьюти блогеры,,https://yt3.ggpht.com/CR5Fr0mKRli1NpBtvMPxIXmO...,https://yt3.ggpht.com/CR5Fr0mKRli1NpBtvMPxIXmO...,https://yt3.ggpht.com/CR5Fr0mKRli1NpBtvMPxIXmO...,Бьюти блогеры,none,2021-11-30T11:21:15Z
3,youtube#searchResult,_mQOJAn6oezgLxtHn5UXRF7x6_8,youtube#channel,UCYnxTYxldRW3wFJIc_6vMTg,2018-03-20T05:05:30Z,UCYnxTYxldRW3wFJIc_6vMTg,Бьюти Блогеры,,https://yt3.ggpht.com/ytc/AL5GRJV7fogwOeRwam_Q...,https://yt3.ggpht.com/ytc/AL5GRJV7fogwOeRwam_Q...,https://yt3.ggpht.com/ytc/AL5GRJV7fogwOeRwam_Q...,Бьюти Блогеры,none,2018-03-20T05:05:30Z
0,youtube#searchResult,sxvnNUa20E3mL2YLxQcBVCvBxmI,youtube#channel,UCW0-JsQvk2drCyC_gt3Pipg,2016-04-13T18:50:40Z,UCW0-JsQvk2drCyC_gt3Pipg,Лисса,Всем привет! Авеми Лисса бьюти канал для девуш...,https://yt3.ggpht.com/sMUsuVWPx1cDR7X6GibF12gL...,https://yt3.ggpht.com/sMUsuVWPx1cDR7X6GibF12gL...,https://yt3.ggpht.com/sMUsuVWPx1cDR7X6GibF12gL...,Лисса,none,2016-04-13T18:50:40Z


In [23]:
# 12

df_search.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 37
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   kind                            328 non-null    object
 1   etag                            328 non-null    object
 2   id.kind                         328 non-null    object
 3   id.channelId                    328 non-null    object
 4   snippet.publishedAt             328 non-null    object
 5   snippet.channelId               328 non-null    object
 6   snippet.title                   328 non-null    object
 7   snippet.description             328 non-null    object
 8   snippet.thumbnails.default.url  328 non-null    object
 9   snippet.thumbnails.medium.url   328 non-null    object
 10  snippet.thumbnails.high.url     328 non-null    object
 11  snippet.channelTitle            328 non-null    object
 12  snippet.liveBroadcastContent    328 non-null    obj

In [66]:
df_search.to_excel(DATA_PATH / 'Channels_search.xlsx', index=False)

In [25]:
# 13
# Загрузка характеристик видео

search_parts = [
    "brandingSettings",
    "contentDetails",
    "contentOwnerDetails",
    "snippet",
    "statistics",
    "topicDetails",
]

# список каналов
channel_ids = list(df_search['id.channelId'])

In [29]:
# 14 Множественный вызов по 50 элементов

df_supplemented = pd.DataFrame()

for i in range(0, len(df_search), 50):
    # границы интервала
    s, e = i, i+50
    #if e > len(df_search): e = len(df_search)

    request = youtube.channels().list(
    part=search_parts,
    maxResults=50,
    id=channel_ids[s:e],
    )
    response = request.execute()

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])
    print(f"Итерация {i // 50} завершена")#

Итерация 0 завершена
Итерация 1 завершена
Итерация 2 завершена
Итерация 3 завершена
Итерация 4 завершена
Итерация 5 завершена
Итерация 6 завершена


In [30]:
# 15 Валидация количества каналов

if len(df_search) == len(df_supplemented): print('Всё ок')
else: print("Не досчитались:", abs(len(df_search) - len(df_supplemented)), "каналов")

Всё ок


In [31]:
# 16
df_channel = df_supplemented.copy(deep=True)
df_channel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 27
Data columns (total 36 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   kind                                                 328 non-null    object
 1   etag                                                 328 non-null    object
 2   id                                                   328 non-null    object
 3   snippet.title                                        328 non-null    object
 4   snippet.description                                  328 non-null    object
 5   snippet.customUrl                                    328 non-null    object
 6   snippet.publishedAt                                  328 non-null    object
 7   snippet.thumbnails.default.url                       328 non-null    object
 8   snippet.thumbnails.default.width                     328 non-null    int64 
 9   

In [32]:
# 17 Преобразование числовых колонок для корректного отображения
numeric_columns = [
    "contentDetails.relatedPlaylists.likes",
    "contentDetails.relatedPlaylists.uploads",
    'statistics.viewCount',
       'statistics.subscriberCount', 'statistics.hiddenSubscriberCount',
       'statistics.videoCount'
]


for num_col in numeric_columns:

    df_channel[num_col] = pd.to_numeric(df_channel[num_col], errors='coerce')


In [33]:
# 18 Отображение числовых характеристик для фильтрации

pd.set_option('float_format', '{:f}'.format)

df_channel[numeric_columns].describe()

,contentDetails.relatedPlaylists.likes,contentDetails.relatedPlaylists.uploads,statistics.viewCount,statistics.subscriberCount,statistics.videoCount
count,0.000000,0.000000,328.000000,328.000000,328.000000
mean,NaN,NaN,41590835.024390,217742.640244,250.582317
std,NaN,NaN,170636457.395853,691658.252380,515.857167
min,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,1306.250000,28.750000,6.750000
50%,NaN,NaN,462090.000000,5685.000000,52.000000
75%,NaN,NaN,12037736.500000,128000.000000,294.500000
max,NaN,NaN,2201759391.000000,9210000.000000,5152.000000


# Фильтрация

1. Количество просмотров - 50% (медиана)
2. Количество видео - 50% (медиана)


In [34]:
# 19 Вывод характеристик для фильтрации

view_median = np.median(df_channel['statistics.viewCount'])
sub_median = np.median(df_channel['statistics.subscriberCount'])

print('Отсев по просмотрам', view_median)
print('Отсев по подписчикам', sub_median)

Отсев по просмотрам 462090.0
Отсев по подписчикам 5685.0


In [35]:
# 20 Статистика после фильтра

df_final_channels = df_channel[(df_channel['statistics.viewCount'] >= view_median) | (df_channel['statistics.subscriberCount'] >= sub_median)]

df_final_channels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173 entries, 0 to 27
Data columns (total 36 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   kind                                                 173 non-null    object 
 1   etag                                                 173 non-null    object 
 2   id                                                   173 non-null    object 
 3   snippet.title                                        173 non-null    object 
 4   snippet.description                                  173 non-null    object 
 5   snippet.customUrl                                    173 non-null    object 
 6   snippet.publishedAt                                  173 non-null    object 
 7   snippet.thumbnails.default.url                       173 non-null    object 
 8   snippet.thumbnails.default.width                     173 non-null    in

In [107]:
# 21 Сохранение

df_final_channels.to_excel(DATA_PATH / f'{query}_channels.xlsx', index=False)